In [1]:
import torch as t
import torch.nn.functional as F
from tqdm import tqdm
from models.Autoencoder import Autoencoder
from torchvision import datasets, transforms

In [34]:
bs = 512
train_ds = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
train_dl = t.utils.data.DataLoader(dataset=train_ds, batch_size=bs, shuffle=True, drop_last=True)

In [35]:
train_ds

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [50]:
train_dl

In [49]:
for i in range(10):
    print(train_ds[i][0].shape, train_ds[i][1])

torch.Size([1, 28, 28]) 5
torch.Size([1, 28, 28]) 0
torch.Size([1, 28, 28]) 4
torch.Size([1, 28, 28]) 1
torch.Size([1, 28, 28]) 9
torch.Size([1, 28, 28]) 2
torch.Size([1, 28, 28]) 1
torch.Size([1, 28, 28]) 3
torch.Size([1, 28, 28]) 1
torch.Size([1, 28, 28]) 4


In [ ]:
for image, label in train_ds[:10]:
    print(image.shape, label)

In [4]:
train_ds[0][0].size()

torch.Size([1, 28, 28])

In [5]:
train_ds[0][1]

5

In [52]:
train_ds[0][0][None].shape

torch.Size([1, 1, 28, 28])

In [54]:
train_ds[0][0].unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [ ]:
train_ds[0][0][None] == train_ds[0][0].unsqueeze(0)

In [7]:
device = t.device('cuda') if t.cuda.is_available() else 'cpu'
# model = Autoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
#                     enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 0, 1], enc_strides=[1, 2, 2, 1],
#                     dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
#                     dec_pads=[1, 0, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
model = Autoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
                    enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 1, 1], enc_strides=[1, 2, 2, 1],
                    dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
                    dec_pads=[1, 1, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
# model.cuda(device)
# model

In [8]:
from torchsummary import summary

In [64]:
summary(model, input_size=(1, 28, 28), batch_size=3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [3, 32, 28, 28]             320
         LeakyReLU-2            [3, 32, 28, 28]               0
       BatchNorm2d-3            [3, 32, 28, 28]              64
           Dropout-4            [3, 32, 28, 28]               0
            Conv2d-5            [3, 64, 14, 14]          18,496
         LeakyReLU-6            [3, 64, 14, 14]               0
       BatchNorm2d-7            [3, 64, 14, 14]             128
           Dropout-8            [3, 64, 14, 14]               0
            Conv2d-9              [3, 64, 7, 7]          36,928
        LeakyReLU-10              [3, 64, 7, 7]               0
      BatchNorm2d-11              [3, 64, 7, 7]             128
          Dropout-12              [3, 64, 7, 7]               0
           Conv2d-13              [3, 64, 7, 7]          36,928
        LeakyReLU-14              [3, 6

In [74]:
model

Autoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU

In [75]:
for name, p in model.named_parameters():
    print(name, p.size())

enc_conv_layers.0.0.weight torch.Size([32, 1, 3, 3])
enc_conv_layers.0.0.bias torch.Size([32])
enc_conv_layers.0.2.weight torch.Size([32])
enc_conv_layers.0.2.bias torch.Size([32])
enc_conv_layers.1.0.weight torch.Size([64, 32, 3, 3])
enc_conv_layers.1.0.bias torch.Size([64])
enc_conv_layers.1.2.weight torch.Size([64])
enc_conv_layers.1.2.bias torch.Size([64])
enc_conv_layers.2.0.weight torch.Size([64, 64, 3, 3])
enc_conv_layers.2.0.bias torch.Size([64])
enc_conv_layers.2.2.weight torch.Size([64])
enc_conv_layers.2.2.bias torch.Size([64])
enc_conv_layers.3.0.weight torch.Size([64, 64, 3, 3])
enc_conv_layers.3.0.bias torch.Size([64])
enc_conv_layers.3.2.weight torch.Size([64])
enc_conv_layers.3.2.bias torch.Size([64])
enc_conv_layers.5.weight torch.Size([2, 3136])
enc_conv_layers.5.bias torch.Size([2])
dec_conv_layers.0.weight torch.Size([3136, 2])
dec_conv_layers.0.bias torch.Size([3136])
dec_conv_layers.2.0.weight torch.Size([64, 64, 3, 3])
dec_conv_layers.2.0.bias torch.Size([64])
de

In [76]:
import numpy as np

In [77]:
np.prod([1,2,3])

6

In [38]:
optimizer = t.optim.Adam(model.parameters(), lr=5e-4, betas=(.9, .99), weight_decay=1e-2)
model.train()

for epoch in tqdm(range(20)):
    if epoch == 10:
        optimizer = t.optim.Adam(model.parameters(), lr=2e-4, betas=(.9, .99), weight_decay=1e-2)
    for i, (data, _) in enumerate(train_dl):
        data = data.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = F.mse_loss(pred, data)
        loss.backward()
        optimizer.step()
        if i % 33 == 0:
            print(loss)


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

tensor(0.4022, grad_fn=<MseLossBackward>)


  0%|                                                                                           | 0/20 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(loss)
t.save(model.state_dict(), 'models/state_dicts/03_01.pth')